# Lecture 3b

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import xlrd
import time


In [3]:
# These functions are to visulize graph inline
from IPython.display import clear_output, Image, display, HTML
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

## Logistic Regression 

In [4]:
# For MNIST Dataset
# logits = X*w + b
# Y_predicted = softmax(logits)
# loss = cross_entropy(Y,Y_predicted)

### Graph 

In [5]:
# Step 1: Read in Data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/data/mnist',one_hot=True)

Extracting /data/mnist\train-images-idx3-ubyte.gz
Extracting /data/mnist\train-labels-idx1-ubyte.gz
Extracting /data/mnist\t10k-images-idx3-ubyte.gz
Extracting /data/mnist\t10k-labels-idx1-ubyte.gz


In [6]:
# Defing Model
# Define parameters 
tf.reset_default_graph()
learning_rate = 0.1
batch_size = 128
n_epochs = 30


#Step 2: 
X = tf.placeholder(tf.float32,[batch_size,784],name='X_placeholder')
Y = tf.placeholder(tf.float32,[batch_size,10],name='Y_placeholder')
#Step 3:
weights = tf.Variable(tf.random_normal(shape=[784,10],stddev=0.01),name='weights') 
bias = tf.Variable(tf.zeros([1,10]),name='bias')

#Step 4:
logits = tf.matmul(X,weights) + bias

#Step 5: 
entropy = tf.nn.softmax_cross_entropy_with_logits(labels=Y,logits=logits,name='loss')
loss = tf.reduce_mean(entropy)
# Step 6:
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

In [7]:
# Run Session
with tf.Session() as sess:
    writer = tf.summary.FileWriter('./graphs/lect_3',sess.graph)
    start_time = time.time()
    sess.run(tf.global_variables_initializer())
    n_batches = int(mnist.train.num_examples/batch_size)
    for i in range(n_epochs):
        total_loss = 0
        for j in range(n_batches):
            X_batch,Y_batch = mnist.train.next_batch(batch_size)
            _,loss_batch = sess.run([optimizer,loss],feed_dict
                                    ={X:X_batch,Y:Y_batch})
            total_loss +=loss_batch
        print('Average loss epoch {0}:{1}'.format(i,total_loss/n_batches))    
    print('Total_time:{0} seconds',format(time.time()-start_time))
# Test The model
    n_batches = int(mnist.test.num_examples/batch_size)
    total_correct_preds = 0
    for i in range(n_batches):
        X_batch, Y_batch = mnist.test.next_batch(batch_size)
        loss_batch,logits_batch = sess.run([loss,logits],feed_dict={X:X_batch,Y:Y_batch})
        probs = tf.nn.softmax(logits_batch)
        correct_preds = tf.equal(tf.argmax(probs,1),tf.argmax(Y_batch,1))
        n_correct_preds = tf.reduce_sum(tf.cast(correct_preds,tf.float32))
        total_correct_preds += sess.run(n_correct_preds)
    print('Accuracy{0}'.format(total_correct_preds/mnist.test.num_examples))    
writer.close()           

Average loss epoch 0:0.5895985464255015
Average loss epoch 1:0.38229307681669444
Average loss epoch 2:0.3491351484627157
Average loss epoch 3:0.3309470740231601
Average loss epoch 4:0.3210917798789231
Average loss epoch 5:0.31225927754159855
Average loss epoch 6:0.3057634740164786
Average loss epoch 7:0.3011438046301995
Average loss epoch 8:0.2975122494783713
Average loss epoch 9:0.29388150850653927
Average loss epoch 10:0.2919832657336633
Average loss epoch 11:0.2877503954620906
Average loss epoch 12:0.28473833161634166
Average loss epoch 13:0.28498443872570156
Average loss epoch 14:0.28234671052797133
Average loss epoch 15:0.2792661496392497
Average loss epoch 16:0.28001140801361946
Average loss epoch 17:0.2785054968196751
Average loss epoch 18:0.2752267465794281
Average loss epoch 19:0.2753134447134736
Average loss epoch 20:0.27359294934775724
Average loss epoch 21:0.2754184245924294
Average loss epoch 22:0.2717709303909526
Average loss epoch 23:0.27183115477120123
Average loss epoc

In [8]:
show_graph(tf.get_default_graph().as_graph_def())

TypeError: '<stripped 5120 bytes>' has type <class 'str'>, but expected one of: ((<class 'bytes'>,),)